In [2]:
1_00_00_000

10000000

In [5]:
from typing import cast
a = 10
b = cast(str, a)
result:int = cast(int, None)

In [2]:
import pandas as pd
import numpy as np
a = pd.DataFrame()
b = pd.DataFrame(np.arange(8))

In [3]:
a+b

,0
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,NaN
7,NaN
